In [1]:
from gssutils import *

In [2]:
scraper = Scraper('https://www.communities-ni.gov.uk/publications/topic/8182?search=%22Northern+Ireland+Housing+Statistics%22&sort_by=field_published_date')
tabs = { tab.name: tab for tab in scraper.distributions[1].as_databaker() } 

In [3]:
next_table = pd.DataFrame()

In [4]:
%%capture

%run "T2.1.py"
next_table = pd.concat([next_table, new_table])
%run "T2.2.py"
next_table = pd.concat([next_table, new_table])
%run "T2.3.py"
next_table = pd.concat([next_table, new_table])
%run "T2.4.py"
next_table = pd.concat([next_table, new_table])
%run "T2.5.py"
next_table = pd.concat([next_table, new_table])

In [5]:
next_table['Period'] = next_table['Period'].str.replace('\.0', '')

In [6]:
next_table['NI Household Description'] = next_table['NI Household Description'].apply(pathify)

In [7]:
next_table['NI Household Energy Status'] = next_table['NI Household Energy'].apply(pathify)

In [8]:
next_table['NI Household Energy Status'] = next_table['NI Household Energy Status'].map(lambda cell: cell.replace('/','-'))

In [9]:
next_table['NI Household Energy Status'] = next_table['NI Household Energy Status'].str.lstrip('-')

In [10]:
def user_perc(x):
    
    if (str(x) ==  '*')  | (str(x) == '..'): 
        
        return 'not-defined'
    else:
        return 'no-marker'
    
next_table['NI Marker'] = next_table.apply(lambda row: user_perc(row['NI Marker']), axis = 1)

In [11]:
next_table = next_table[['Period','NI Household Description','NI Household Energy Status','Unit','Value','Measure Type','NI Marker']]

In [15]:
from pathlib import Path
out = Path('out')
out.mkdir(exist_ok=True)
next_table.drop_duplicates().to_csv(out / 'observations.csv', index = False)

In [23]:
import numpy as np
from os import environ

nme = "Northern Ireland Housing Statistics Energy"
pnme = pathify(nme)
scraper.dataset.family = 'affordable-housing'
print(nme)
print(pnme)

Northern Ireland Housing Statistics Energy
northern-ireland-housing-statistics-energy


In [24]:
scraper.dataset.title = nme
scraper.set_dataset_id(f'{pathify(environ.get("JOB_NAME", ""))}/{pnme}')
with open(out / 'observations.csv-metadata.trig', 'wb') as metadata:
    scraper.dataset.license = "http://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/"
    metadata.write(scraper.generate_trig())

In [25]:
csvw = CSVWMetadata('https://gss-cogs.github.io/family-affordable-housing/reference
                   
csvw.create(out / 'observations.csv', out / 'observations.csv-schema.json')